## Entendendo como extrair informações sobre repositórios no github

[Search](https://developer.github.com/v3/search/)
[Searching for repositories](https://help.github.com/en/articles/searching-for-repositories#search-by-repository-name-description-or-contents-of-the-readme-file)


Lista de qualifiers:  https://help.github.com/en/articles/searching-code
Documentação para search: https://developer.github.com/v3/search/

In [197]:
import requests
import pandas as pd

Montando url de busca para os repositórios com maior quantidade de estrelas.
Usando como consulta stars:>1 para indicar que são todos os repositorios acima de 1 estrela, o que não tem impacto negativo para a busca pois existem certa de 3.305.358 repositórios e só vamos analisar 2.500.

In [198]:
q = 'q=stars:>1'
sort = '&sort=stars&order=desc'
url_base = 'https://api.github.com/search/repositories?'
url_final = url_base+q+sort
url_final

'https://api.github.com/search/repositories?q=stars:>1&sort=stars&order=desc'

In [199]:
colunas=['owner_type', 
           'owner_url',
           'owner_html_url',
           'html_url',
           'url',
           'fork',
           'created_at',
           'updated_at',
           'size',
           'stargazers_count',
           'language',
           'has_issues',
           'has_wiki',
           'forks_count',
           'forks',
           'open_issues',
           'watchers']

In [200]:
resultados = pd.DataFrame(columns=['owner_type', 
                                   'owner_url',
                                   'owner_html_url',
                                   'html_url',
                                   'url',
                                   'fork',
                                   'created_at',
                                   'updated_at',
                                   'size',
                                   'stargazers_count',
                                   'language',
                                   'has_issues',
                                   'has_wiki',
                                   'forks_count',
                                   'forks',
                                   'open_issues',
                                   'watchers',
                                   'commits',
                                   'contributors',
                                   'readme',])
resultados.head()

,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,stargazers_count,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme


In [201]:
def add_resultado(item):
    df = pd.DataFrame([[
                        item.get('owner').get('type', None),
                        item.get('owner').get('url', None),
                        item.get('owner').get('html_url', None),
                        item.get('html_url', None),
                        item.get('url', None),
                        item.get('fork', None),
                        item.get('created_at', None),
                        item.get('updated_at', None),
                        item.get('size', None),
                        item.get('stargazers_count', None),
                        item.get('language', None),
                        item.get('has_issues', None),
                        item.get('has_wiki', None),
                        item.get('forks_count', None),
                        item.get('forks', None),
                        item.get('open_issues', None),
                        item.get('watchers', None)
                    ]], columns=colunas)

    return df    

In [202]:
def extrair_dados(data, resultados):
    for item in data.get('items', None):
        resultados = pd.concat([resultados, add_resultado(item)], ignore_index=True, sort=False)
        
    return resultados

In [203]:
def get_total_paginas(data):
    
    itens_por_pagina = len(data.get('items'))
    total_paginas = data.get('total_count') // itens_por_pagina # opreração de div em python
    total_paginas

    print('Total de registros:{0} , Registros por página:{1}, Total de Páginas:{2}'.format(
        data.get('total_count'), itens_por_pagina, total_paginas))
    
    return total_paginas        

In [207]:
def percorrendo_paginas(url):
    
    resultados = pd.DataFrame(columns=['owner_type', 
                                   'owner_url',
                                   'owner_html_url',
                                   'html_url',
                                   'url',
                                   'fork',
                                   'created_at',
                                   'updated_at',
                                   'size',
                                   'stargazers_count',
                                   'language',
                                   'has_issues',
                                   'has_wiki',
                                   'forks_count',
                                   'forks',
                                   'open_issues',
                                   'watchers',
                                   'commits',
                                   'contributors',
                                   'readme',])
    
    results = requests.get(url)    
    data = dict(results.json())
    extrair_dados(data, resultados)

    # iteracoes = get_total_paginas(data)
    iteracoes = 84 # Para pegar em torno de 2500 registros, semelhante ao artigo original
    
    for iteracao in range(1, 10):
        print('Extraindo página:{0}'.format(iteracao))
        
        header = dict(results.links)
        next_url = header.get('next').get('url')
        
        results = requests.get(next_url)
        print('Status:{0}'.format(results))
        
        data = dict(results.json())
        print('Data keys:{0}'.format(data.keys()))
            
        resultados = extrair_dados(data, resultados)
        
    return resultados

In [208]:
resultados = percorrendo_paginas(url_final)

Extraindo página:1
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:2
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:3
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:4
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:5
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:6
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:7
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:8
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])
Extraindo página:9
Status:<Response [200]>
Data keys:dict_keys(['total_count', 'incomplete_results', 'items'])


In [209]:
resultados.describe()

,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,stargazers_count,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme
count,270,270,270,270,270,270,270,270,270,270,232,270,270,270,270,270,270,0.0,0.0,0.0
unique,2,252,252,270,270,1,270,258,269,267,27,2,2,269,269,222,267,0.0,0.0,0.0
top,Organization,https://api.github.com/users/apache,https://github.com/apache,https://github.com/gitlabhq/gitlabhq,https://api.github.com/repos/godotengine/godot,False,2014-10-16T18:29:12Z,2019-06-25T15:10:21Z,440,22225,JavaScript,True,True,3445,3445,7,22225,NaN,NaN,NaN
freq,170,4,4,1,1,270,1,5,2,2,86,253,163,2,2,4,2,NaN,NaN,NaN


In [210]:
resultados.to_csv('../dados/repositorios.csv', index=False)

In [211]:
resultados.columns.values

array(['owner_type', 'owner_url', 'owner_html_url', 'html_url', 'url',
       'fork', 'created_at', 'updated_at', 'size', 'stargazers_count',
       'language', 'has_issues', 'has_wiki', 'forks_count', 'forks',
       'open_issues', 'watchers', 'commits', 'contributors', 'readme'], dtype=object)